In [2]:
import pandas as pd
import numpy as np
import ibmdata

In [29]:
DAYSBACK = 365
QUERY = f"""
SELECT 
    wafer_Id, family_Code, unitcell_X as kerf_X, unitcell_Y as kerf_Y,
    --tw.Calcdefs AS Sail, 
    CASE 
        WHEN RIGHT(parm_Label,5) LIKE 'Yield' THEN tw.testProgramName||'_0p75_SkipTest'
        WHEN LENGTH(parm_Label) = 37 THEN tw.testProgramName||'_0p75'||RIGHT(parm_Label,11) 
        ELSE tw.testProgramName||'_0p75'||RIGHT(parm_Label,12)
    END AS label,
    100.0-parmYield as yield_loss
FROM DMIW.ChipParmFactR cpfr
    JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = cpfr.testParmKey
    JOIN DMIW_SYSTEMS.Geography g ON g.geographyKey = cpfr.geographyKey
    JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = cpfr.testedWaferKey
WHERE Last_test_date >= (current date - {DAYSBACK} days)
    AND Tech_id = '7HPP' AND Level = 'H2' AND tw.Calcdefs IN ('SSL21', 'SSL31')
    AND cpfr.parmValue IS NOT null AND ABS(parmValue) < 1e25 
    AND (UCASE(SUBSTR(tp.parm_Label,10,17)) = 'SSCANLATCH0P75VFM' OR UCASE(SUBSTR(tp.parm_Label,10,14)) = '0P75V_SKIPTEST')
    AND ((unitcell_X - 1) || '_' || unitcell_Y NOT IN ('5_13','6_13','7_13','3_12','4_12','8_12','9_12','2_11','10_11','1_10','1_9','11_9','1_5','11_5','1_4','2_3','10_3','3_2','4_2','8_2','9_2','5_1'))
    --AND parmYield < 100
    AND family_Code = 'X2'
    AND (UCASE(SUBSTR(tp.parm_Label,5,1)) = UCASE(RIGHT(tw.testProgramName,1)))
    --AND wafer_id = 'AZ6B5-19'
ORDER BY lot_Id, wafer_Id, parm_Label
"""

In [21]:
df = ibmdata.isdw.query(QUERY)

In [22]:
df

,wafer_id,family_code,kerf_x,kerf_y,label,yield_loss
0,A290V-01,X2,7,8,SSL2_0p75_SkipTest,0.00
1,A290V-01,X2,6,4,SSL2_0p75_SkipTest,0.00
2,A290V-01,X2,11,6,SSL2_0p75_SkipTest,0.00
3,A290V-01,X2,2,8,SSL2_0p75_SkipTest,0.00
4,A290V-01,X2,7,3,SSL2_0p75_SkipTest,0.00
...,...,...,...,...,...,...
25459976,AZD3F-15,X2,6,10,SSL3_0p75_block9_so5,0.00
25459977,AZD3F-15,X2,8,2,SSL3_0p75_block9_so5,0.00
25459978,AZD3F-15,X2,10,8,SSL3_0p75_block9_so5,0.00
25459979,AZD3F-15,X2,7,9,SSL3_0p75_block9_so5,0.00


In [23]:
pvt_df = df.pivot(index=['wafer_id','family_code','kerf_x','kerf_y'], columns='label', values='yield_loss').reset_index(drop = False)

In [26]:
pvt_df

label,wafer_id,family_code,kerf_x,kerf_y,SSL2_0p75_SkipTest,SSL2_0p75_block10_so0,SSL2_0p75_block10_so1,SSL2_0p75_block10_so2,SSL2_0p75_block10_so3,SSL2_0p75_block10_so4,...,SSL3_0p75_block8_so2,SSL3_0p75_block8_so3,SSL3_0p75_block8_so4,SSL3_0p75_block8_so5,SSL3_0p75_block9_so0,SSL3_0p75_block9_so1,SSL3_0p75_block9_so2,SSL3_0p75_block9_so3,SSL3_0p75_block9_so4,SSL3_0p75_block9_so5
0,A290V-01,X2,2,6,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,A290V-01,X2,2,7,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,A290V-01,X2,2,8,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,A290V-01,X2,3,4,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,A290V-01,X2,3,5,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75645,AZD3F-15,X2,11,9,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
75646,AZD3F-15,X2,11,10,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
75647,AZD3F-15,X2,12,6,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
75648,AZD3F-15,X2,12,7,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [28]:
pvt_df.to_csv('/Users/acyang@us.ibm.com/Desktop/Wafer-Spacial-Analysis/Sail_zADD2.csv')